In [ ]:
%%shell
git clone --recurse-submodules https://github.com/castorini/pyserini.git
cd pyserini/tools && cd eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../..
pip install pyserini faiss-cpu -qq
pip install transformers

Cloning into 'pyserini'...
remote: Enumerating objects: 7659, done.
remote: Counting objects: 100% (7659/7659), done.
remote: Compressing objects: 100% (2081/2081), done.
remote: Total 7659 (delta 5608), reused 7477 (delta 5539), pack-reused 0
Receiving objects: 100% (7659/7659), 4.40 MiB | 21.15 MiB/s, done.
Resolving deltas: 100% (5608/5608), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pyserini/tools'...
remote: Enumerating objects: 806, done.        
remote: Counting objects: 100% (563/563), done.        
remote: Compressing objects: 100% (482/482), done.        
remote: Total 806 (delta 106), reused 527 (delta 80), pack-reused 243        
Receiving objects: 100% (806/806), 139.32 MiB | 25.68 MiB/s, done.
Resolving deltas: 100% (190/190), done.
Submodule path 'tools': checked out '7b84f773225b5973b4533dfa0aa18653409a6146'
trec_eval.9.0.4/
trec_eval.9.0.4/m_prefs_pair.c
trec_eval.9.0.4/m_ndcg_p.c
trec_ev

In [ ]:
import json
from tqdm.auto import tqdm

In [ ]:
# Corpus with all passages
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
!gzip -cd corpus.jsonl.gz > corpus.jsonl

--2023-06-28 18:45:57--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
Resolving huggingface.co (huggingface.co)... 65.9.86.62, 65.9.86.57, 65.9.86.79, ...
Connecting to huggingface.co (huggingface.co)|65.9.86.62|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/e9e97686e3138eaff989f67c04cd32e8f8f4c0d4857187e3f180275b23e24e85?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27corpus.jsonl.gz%3B+filename%3D%22corpus.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1688237158&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvZTllOTc2ODZlMzEzOGVhZmY5ODlmNjdjMDRjZDMyZThmOGY0YzBkNDg1NzE4N2UzZjE4MDI3NWIyM2UyNGU4NT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzcG9u

In [ ]:
!mkdir corpus_original

corpus = {}
with open("corpus_original/corpus_original.jsonl", "w") as fout:
    with open("corpus.jsonl", "r") as fin:
        for line in fin:
            doc = json.loads(line)
            corpus[doc["_id"]] = f"{doc['title']} . {doc['text']}"
            doc_dict = {
                "id": doc["_id"],
                "contents": f"{doc['title']} . {doc['text']}"
            }
            fout.write(json.dumps(doc_dict) + "\n")

In [ ]:
# original index
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input corpus_original \
  --index original_index \
  --language en\
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storeRaw

2023-06-28 18:46:10,857 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2023-06-28 18:46:10,861 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2023-06-28 18:46:10,862 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2023-06-28 18:46:10,862 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: corpus_original
2023-06-28 18:46:10,862 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2023-06-28 18:46:10,864 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2023-06-28 18:46:10,865 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2023-06-28 18:46:10,870 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: en
2023-06-28 18:46:10,870 INFO  [main] index.IndexCollection (IndexC

In [ ]:
# Topics with all queries
!wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
!gzip -dc queries.jsonl.gz > queries.jsonl

--2023-06-28 18:46:34--  https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/queries.jsonl.gz
Resolving huggingface.co (huggingface.co)... 65.9.86.57, 65.9.86.79, 65.9.86.62, ...
Connecting to huggingface.co (huggingface.co)|65.9.86.57|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/a8/10/a810e88b0e7b233be82b89c1fa6ec2d75efc6d55784c2ada9dcac8434a634f3a/9eadcc2cdf140addc9dae83648bb2c6611f5e4b66eaed7475fa5a0ca48eda371?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27queries.jsonl.gz%3B+filename%3D%22queries.jsonl.gz%22%3B&response-content-type=application%2Fgzip&Expires=1688237195&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2E4LzEwL2E4MTBlODhiMGU3YjIzM2JlODJiODljMWZhNmVjMmQ3NWVmYzZkNTU3ODRjMmFkYTlkY2FjODQzNGE2MzRmM2EvOWVhZGNjMmNkZjE0MGFkZGM5ZGFlODM2NDhiYjJjNjYxMWY1ZTRiNjZlYWVkNzQ3NWZhNWEwY2E0OGVkYTM3MT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSomcmVzc

In [ ]:
# Qrels with all relevances
!wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv

--2023-06-28 18:46:35--  https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
Resolving huggingface.co (huggingface.co)... 65.9.86.57, 65.9.86.79, 65.9.86.62, ...
Connecting to huggingface.co (huggingface.co)|65.9.86.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 980831 (958K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 957.84K  2.86MB/s    in 0.3s    

2023-06-28 18:46:35 (2.86 MB/s) - ‘test.tsv’ saved [980831/980831]



In [ ]:
topics = []

with open("queries.jsonl", "r") as f:
  for line in f:
    query = json.loads(line)
    topics.append((query["_id"], query["text"]))

with open("qrels.tsv", "w") as fout:
  with open("test.tsv", "r") as fin:
    for idx, line in enumerate(fin):
      if idx != 0:
        qid, doc_id, relevance = line.strip().split("\t")
        fout.write(f"{qid}\t0\t{doc_id}\t{relevance}\n")

def get_run_bm25(path, topics, top_k, searcher):
    with open(path, 'w') as f:
        for id, query in tqdm(topics, desc="Running queries"):
            hits = searcher.search(query, top_k)
            for idx, hit in enumerate(hits):
              f.write(f"{id}\tQ0\t{hit.docid}\t{idx+1}\t{hit.score}\tpedro\n")

In [ ]:
from pyserini.search.lucene import LuceneSearcher

In [ ]:
searcher = LuceneSearcher('/content/original_index')
get_run_bm25('original-bm25.tsv', topics, 100, searcher)

Running queries:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -l 2 qrels.tsv original-bm25.tsv

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '-l', '2', 'qrels.tsv', 'original-bm25.tsv']
Results:
ndcg_cut_10           	all	0.5947


In [ ]:
from transformers import AutoTokenizer, LongT5ForConditionalGeneration

In [ ]:
pretrained_model_name = "pedrogengo/listwise_longt5_1k_msmarco"

tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")
model = LongT5ForConditionalGeneration.from_pretrained(pretrained_model_name)

In [ ]:
_ = model.to("cuda")
_ = model.eval()

In [ ]:
def generate_permutation_text(query, passages):
  text = ["You are an intelligent assistant that can rank passages based on their relevancy to the query.",
          f"The following are {len(passages)} passages, each indicated by number identifier []. I can rank them based on their relevance to query: {query}",]

  for idx, passage in enumerate(passages):
    text.append(f"[doc_{idx+1}] {passage}")

  text.append(f"The search query is: {query}")
  text.append(f"I will rank the {len(passages)} passages above based on their relevance to the search query. The passages will be listed in descending order using identifiers, and the most relevant passages should be listed first, and the output format should be [] > [] > etc, e.g., [doc_1] > [doc_2] > etc.")
  text.append(f"The ranking results of the {len(passages)} passages (only identifiers) is:")
  return "\n".join(text)

In [ ]:
def generate_prediction(model, prompt):
    model_inputs = tokenizer([prompt], padding="longest", return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(model_inputs, max_length=200, do_sample=False)
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    try:
      output = [docid.strip()[1:-1] for docid in pred.split(">")]
      assert len(output) == 9
      for i in range(1,10):
        assert f"doc_{i}" in output
      return output
    except:
      print("returning bm25 prediction")
      return ["doc_9", "doc_8", "doc_7", "doc_6", "doc_5", "doc_4", "doc_3", "doc_2", "doc_1"]

In [ ]:
def get_run_listwise(path, topics, top_k, stride, searcher, model):
    window = 9
    with open(path, 'w') as f:
        for id, query in tqdm(topics, desc="Running queries"):
            hits = searcher.search(query, top_k)
            hits = list(reversed(hits))
            reranked = [0] * top_k
            for idx in range(0, len(hits) - stride, stride):
                if idx + window > len(hits):
                  idx = len(hits) - window
                passages = [json.loads(hit.raw)["contents"] for hit in hits[idx:idx+window]]
                mapping = {f"doc_{i+1}": [hit.docid, hit.score] for i, hit in enumerate(hits[idx:idx+window])}
                input_text = generate_permutation_text(query, passages)
                prediction = generate_prediction(model, input_text)

                for j, pred in zip(range(idx, idx + window), reversed(prediction)):
                  reranked[top_k - j - 1] = mapping[pred]

                new_order = []
                for docid in reversed(prediction):
                  new_order.append(hits[idx:idx+window][int(docid[-1]) - 1])
                hits[idx:idx+window] = new_order
            for idx, hit in enumerate(reranked):
              f.write(f"{id}\tQ0\t{hit[0]}\t{idx+1}\t{top_k - idx}\tpedro\n")

In [ ]:
get_run_listwise("listwise-longt5.tsv", topics, 100, 4, searcher, model)

Running queries:   0%|          | 0/50 [00:00<?, ?it/s]

returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction
returning bm25 prediction


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -l 2 qrels.tsv listwise-longt5.tsv

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '-l', '2', 'qrels.tsv', 'listwise-longt5_corrected.tsv']
Results:
ndcg_cut_10           	all	0.5536
